# TP Supervised Learning

Ce notebook est basé sur le numerical tour de *Gabriel Peyré* qui introduit les notions essentielles à la regression linéaire et logistique.

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

Fonctions utiles :

In [32]:
#  convert to a column vector
def MakeCol(y): return y.reshape(-1,1)
#  convert to a row vector
def MakeRow(y): return y.reshape(1,-1)
# find non zero/true elements
def find(x): return np.nonzero(x)[0]

## Exercice 1 : Linear Regression

### Data Simulation

Tout d'abord on simule un problème de régression.

In [33]:
from sklearn import datasets
X, y, coef = datasets.make_regression(n_samples = 1000, n_features = 100,
                                n_informative = 4, n_targets = 1,
                                noise = 10.0, coef = True)

**Q: Que fait le code ci-dessus, que font les paramètres ?**

 Ce code génère un ensemble de données synthétiques pour une tâche de régression avec 100 échantillons, 10 caractéristiques dont 4 sont informatives, une seule cible à prédire, et ajoute du bruit gaussien. Les **données d'entrée**, les **valeurs cibles** et les **coefficients** utilisés pour générer les données sont stockés dans les variables **X**, **y** et **coef**, respectivement.

**Q : Séparer le dataset en un ensemble d'apprentissage et de test.**

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test= train_test_split(X, y, test_size=0.5)

**Q : Pourquoi n'utilise-t-on pas d'ensemble de validation ici ?**

Il n'y a pas d'hyperparamètre

###  Least square solution

On rappelle que l'estimateur des moindres carrés est donné par : $\hat{\beta} = (X^T X)^{-1} X^Ty $

**Q: Coder une fonction qui permet de calculer une estimation de celui-ci à partir des données.**

In [35]:
def test(X,y) :
    B = np.linalg.inv(np.dot(np.transpose(X),X))
    B = np.dot(B,np.transpose(X))
    return np.dot(B,y)

def reg_OLS(X,y):
    return np.linalg.inv(np.transpose(X)@X)@np.transpose(X)@y

assert(test(X,y).all() == reg_OLS(X,y).all())
print(reg_OLS(X,y))

[-3.33569890e-02 -2.52981441e-01  4.43709718e-01  3.86399348e-02
 -3.65428815e-02  2.46297038e-01  1.39226710e-01  5.66586987e+01
  3.11710899e-01  2.32585207e-01  3.73198226e-01  1.75951930e-03
 -2.31949485e-01 -3.40718915e-01 -1.17619774e-02 -5.97715151e-01
  2.22224850e-01 -3.78328271e-01  5.51583625e-02 -2.34301198e-01
 -2.28757937e-01 -3.01840389e-01  4.47393724e-02  3.34591494e+01
 -4.10297981e-01 -6.82122944e-01 -5.52180889e-02 -8.89267777e-01
 -1.80490197e-01 -1.96511276e-01  2.41780684e-02  6.16258269e-02
  4.75574822e-01 -3.43298179e-01 -5.08133178e-01  4.30182767e-01
 -5.36012709e-02  5.00088111e-02 -2.14757185e-02 -1.73795894e-01
  9.32696484e-02 -1.04044876e-02  1.13615189e-01 -2.99454066e-01
  3.79115833e-01  2.24143107e-01 -1.95487694e-01 -6.16605551e-02
 -1.50225533e-01  1.52380280e-01 -8.44406130e-02 -7.76860634e-02
 -4.65765799e-01 -4.52521221e-01  3.50078229e-01 -2.56971137e-01
  2.76203310e-01 -5.91441735e-01 -2.53723090e-01 -2.14779898e-01
 -2.35780794e-01 -4.87354

**Q: Donner un avis sur les valeurs des paramètres. Que constate-t-on et pourquoi ?**

**Q: Créer une fonction `reg_lin` qui permet de prédire les labels `y` pour de nouvelles données `X`.**

In [36]:
def reg_lin(X_train,X_test,y):
    coeff = reg_OLS(X_train,y)
    y_pred = X_test @ coeff
    return y_pred

Y_pred = reg_lin(X_train,X_test,Y_train)
print(Y_pred)
#données proche de 0 si on augmente n_samples (=100), n_features (=10)

[ 1.75474516e+02 -5.59104303e+01 -7.05235255e+01 -7.69947423e+01
 -3.95933020e+01  4.51945766e+01 -1.15105726e+02 -5.17670361e+01
  6.99269511e+01 -3.43313313e+02  1.11750791e+02 -1.68198317e+02
 -1.10046444e+02 -5.87476683e+01 -1.23156371e+02  7.39474603e+01
  2.98367182e+01  2.29635357e+02 -5.42509966e+01  1.10865890e+02
  3.53414835e+01  7.03760715e+01 -2.16928485e+01  1.54832791e+02
 -1.47367070e+02 -1.11604030e+01  9.83694631e+01 -8.17780096e+01
 -1.65774677e+02  1.62410840e+02  6.21532528e+01  2.48592422e+02
 -1.35976652e+02  4.18065298e+01 -2.28902638e+02  4.09658978e+01
 -5.21562959e+01  1.75270438e+02 -1.07347814e+02 -1.92317759e+00
 -1.53842466e+02 -1.71741219e+02  3.05620659e+02 -9.88566655e+01
  5.15050544e+01 -4.49573307e+01  1.21117438e+02 -7.04788153e+01
  4.25337474e+01  2.79954370e+01 -1.11939110e+02  1.14039243e+02
 -8.13948884e+01 -7.05557286e+01  4.21520345e+01  1.88381255e+01
  1.88895560e+02 -1.74427974e+01 -1.97940183e+01  5.50100120e+01
 -9.88563176e+01  1.17530

**Q : Tester sur l'ensemble de test. Quel taux d'erreur avez-vous ? Que pouvez-vous dire ?**

In [37]:
def root_mean_squared_error(y_true,y_pred):
    return math.sqrt(((y_true-y_pred)**2).mean())

print(np.var(Y_pred))
print(root_mean_squared_error(Y_train,Y_pred))

13694.754803677026
166.77374646487652


###  Descente de gradient

On rappelle que l'objectif est de minimiser $f(\beta) = \frac{1}{2} \| X\beta - y \|^2$

**Q: Coder la fonction de perte (loss) `f` qui prend en paramètres la matrice X, y et $\beta$.**

In [38]:
def f(X,y,B):
    return 0.5*np.linalg.norm(X@B-y)**2

On rappelle aussi que le gradient de $f$ est : $\nabla f(\beta) = X^T(X\beta - y)$.

**Q: Coder la fonction  `grad_f` qui prend en paramètre la matrice X, y et $\beta$**

In [39]:
def grad_f(X,y,B):
    return np.transpose(X)@(X@B-y)

Maintenant, codons la descente de gradient, on rappelle que le passage de l'algorithme de l'étape $m$ à $m+1$ est donné par :
$$
\beta^{(m+1)} = \beta^{(m)} - \tau \nabla f(\beta^{(m)}),
$$
où $\tau$ est le pas de la descente.

**Q : Coder la fonction `Reg_lin_desc_grad` qui prend en entrée :`X`,`y`, `w` (initialisation de beta), `tau`, `n_iter` (le nombre d'itérations maximum), `tol` (la tolérance de la convergence).**

Dans un second temps, cette fonction doit aussi afficher l'evolution de l'erreur d'entraînement comme une fonction du nombre d'itération.

In [41]:
def Reg_lin_desc_grad(X,y,w,tau = 1/np.linalg.norm(X,2)**2 ,n_iter=50,tol = 1e3,plot=True):
    B = [w]
    for i in range(1,n_iter+1):
        B.append(B[i] - tau*grad_f(X,y,B[i]))
        if (abs(B[i]-B[i-1]) < tol):
            return B[-1]
    return B[-1]

print(Reg_lin_desc_grad(X,y,np.ones(10)))

IndexError: list index out of range

La variable `tau` définie en amont est donnée par un critère du controle de norme, il ne faut pas que le pas de l'algorithme dépasse  :
$$
\tau_{max} = \dfrac{2}{\| X X^T\|_{op}},
$$
avec $\| .\|_{op}$ est la valeur propre maximal.

**Q : Tester pour différentes valeurs de $\tau$, que constatez-vous ?** (Convergence, estimation, etc.)

###  Scikit-Learn

In [ ]:
from sklearn.linear_model import LinearRegression
Model_LinearReg = LinearRegression()

**Q: Que contient cet objet ? Comment mettre à jour ses poids ? Que dire des paramètres ?**

###  OLS / Gradient descent / Scikit-learn (learning time)

Cette partie dépendra grandement de votre machine et de votre implémentation, toutefois elle permettra de vous faire une idée quant au choix de la méthode à privilégier.

In [ ]:
from time import time 
t1 = time() 
#Quelque chose
t2 = time()
elapsed = t2 - t1 
print('Elapsed time is %f seconds.' % elapsed) 

**Q : Que fait le code ci-dessus ?**

**Q: Afficher l'évolution du temps de calcul en faisant varier le nombre d'individus dans le dataset  N = 100 : 100000 (pas logartihmique) et en laissant fixé p = 40.** 

**Q: Afficher l'évolution du temps de calcul en faisant varier le nombre de variables dans le dataset p = 40:5000 (pas logarithmique) et en laissant le nombre d'individus fixé N = 10000.**

###  Gradient descent / Scikit-learn (performances)

**Q: Comparer les performances de prédiction sur l'ensemble de test. Y a-t-il des différences ? Si oui, pourquoi ?**

## Exercice 2 : Logistic Regression

### Data Simulation

In [ ]:
n = 1000 # number of sample
p = 2 # dimensionality
omega = np.array([1,.5])*2.5 # offset 
n1 = int(n/2)
X = np.vstack(( np.random.randn(n1,2), np.random.randn(n1,2)+np.ones([n1,1])*omega ))
y = np.vstack(( np.ones([n1,1]), -np.ones([n1,1]) ))

In [ ]:
I = find(y==-1)
J = find(y==1)
plt.clf
plt.plot(X[I,0], X[I,1], '.')
plt.plot(X[J,0], X[J,1], '.')
plt.axis('equal');

###  Descente de gradient

On rappelle que l'objectif de minimiser $g(\beta) = \dfrac{1}{n} \sum_{i=1:n}  \log(1 + e^{-yi \langle X_i , \beta \rangle})$.

**Q: Coder la fonction de loss `g` qui prend en paramètre la matrice X, y et $\beta$.**

In [ ]:
def g():
    pass

On rappelle aussi que le gradient de $g$ est : $$\nabla g(\beta) = \dfrac{1}{n} X^T y \odot \sigma(- y \ \odot <X,\beta>),$$
où $\odot$ est le produit élément par élément (* en python) et $\sigma$ est la fonction sigmoide.

**Q: Coder la fonction  `grad_g` qui prend en paramètre la matrice X, y et $\beta$.**

In [ ]:
def grad_g():
    pass

Maintenant, codons la descente de gradient. On rappelle que le passage de l'algorithme de l'étape $m$ à $m+1$ est donné par :
$$
\beta^{(m+1)} = \beta^{(m)} - \tau \nabla g(\beta^{(m)}),
$$
où $\tau$ est le pas de la descente.

**Q : Coder la fonction `Reg_log_desc_grad` qui prend en entrée :`X`,`y`, `w` (initialisation de beta), `tau`, `n_iter` (le nombre d'itérations maximum), `tol` (la tolérence de la convergence).**

Dans un second temps, cette fonction doit aussi afficher l'évolution de l'erreur d'entraînement comme une fonction du nombre d'itération.

In [ ]:
def Reg_log_desc_grad(X,y,w,tau = 1/np.linalg.norm(X,2)**2 ,n_iter=50,tol = 1e3,plot=True):
    pass

La variable `tau` définie en amont est donnée par un critère du contrôle de la norme, il ne faut pas que le pas de l'algorithme dépasse  :
$
\tau_{max} = \dfrac{2}{ 1/4\| X \|_{op}^2},
$
avec $\| .\|_{op}$ est la valeur propre maximale.

**Q : Tester différentes valeurs de $\tau$, que constatez-vous ?** (Convergence, estimation, ...)

###  Scikit-Learn

In [ ]:
from sklearn.linear_model import LogisticRegression
Model_LogReg = LogisticRegression()

**Q: Que contient cet objet ? Comment mettre à jour ses poids ? Que dire des paramètres ?**

###  Gradient descent / Scikit-learn (learning time)

Cette partie dépendra grandement de votre machine et de votre implémentation, toutefois elle permettra d'avoir des idées de quand utiliser un algorithme plutôt que l'autre.

On pose $\beta^\star_j = (-1)^{j-1} \exp(-(j-1)/10)$.

In [ ]:
n_features = 40
n_samples = 2000

%matplotlib inline

idx = np.arange(n_features)
params = (-1) ** idx  * np.exp(-idx / 10.)
params[20:] = 0.
plt.stem(params)
plt.title("Parameters")

In [ ]:
import numpy as np
import copy
def sigmoid(a):
    x = copy.deepcopy(a)
    if(x.size ==1) :
        if x >= 0:
            return 1.0 / (1.0 + np.exp(-x))
        else : 
            # Utilisez la formule sigmoid(x) = 1 - sigmoid(-x) pour x < 0
            return np.exp(x) / (1.0 + np.exp(x))
        
        
    neg_part = find(x<=0)
    pos_part = find(x>0)
    if pos_part.size >= 0:
        x[pos_part] = 1.0 / (1.0 + np.exp(-x[pos_part]))
        
    if neg_part.size >= 0: 
        x[neg_part] = np.exp(x[neg_part]) / (1.0 + np.exp(x[neg_part]))
    
    return x

In [ ]:
from numpy.random import multivariate_normal
from scipy.linalg.special_matrices import toeplitz
from numpy.random import binomial
def simu_logreg(n_samples,params=params,rho=0.1):
    """ simulation in a logistic regression model 
    
    Parameters
    ----------
    coefs : `numpy.array`, shape=(n_features,)
        Coefficients of the model
    n_samples : `int`, 
        Number of samples to simulate
    rho : `float`, default=0.1
        Correlation of the features
    Returns
    -------
    X : `numpy.ndarray`, shape=(n_samples, n_features)
    Simulated features matrix. It samples of a centered Gaussian
    vector with covariance given by the Toeplitz matrix
    y : `numpy.array`, shape=(n_samples,)
                 Simulated labels
    """
    n_features = params.size
    
    cov = toeplitz(rho ** np.arange(0, n_features))
    
    features = multivariate_normal(np.zeros(n_features), cov, size=n_samples)
    
    pis = sigmoid(features.dot(params))
    
    labels = 2 * ( binomial(1,pis, n_samples) - 1)
    
    return((features,labels))

In [ ]:
X_perfs,y_perfs = simu_logreg(2000,params,rho = 0.1)
print(y_perfs.shape)
print(X_perfs.shape)

**Q: Que fait le code ci-dessus ? Détailler.**

**Q: Afficher l'évolution du temps de calcul en fonction du nombre d'individus dans le dataset  N = 100 : 100000 et p = 40** 

**Q: Afficher l'évolution du temps de calcul en fonction de variables dans le dataset  N = 10000 et p = 40:5000**

**Q: Afficher l'évolution du temps de calcul en faisant varier le nombre d'individus dans le dataset  N = 100 : 100000 (pas logartihmique) et en laissant fixé p = 40.**

**Q: Afficher l'évolution du temps de calcul en faisant varier le nombre de variables dans le dataset p = 40:5000 (pas logarithmique) et en laissant le nombre d'individus fixé N = 10000.**

###  Gradient descent / Scikit-learn (performances)

**Q: Comparer les performances de prédiction sur l'ensemble de test. Y a-t-il des différences ? Si oui, pourquoi ?**

###  Frontières de décision

En utilisant les fonctions de régression logistique (au choix : celle que vous avez implémentée ou celle de scikit-learn) on va maintenant tracer les frontières de décisions de notre modèle.

On génère une grille de points en 2D.

In [ ]:
q = 201
tx = np.linspace( X[:,0].min(), X[:,0].max(),num=q) 
ty = np.linspace( X[:,1].min(), X[:,1].max(),num=q) 
[B,A] = np.meshgrid( ty,tx )
G = np.vstack([A.flatten(), B.flatten()]).transpose()

**Q: Calculer les probabilités de classe associées à chacun des vecteurs composant la grille.**

In [ ]:
proba_pred = ... 

In [ ]:
proba_class1 = proba_pred[:,0].reshape(q,q)

In [ ]:
plt.clf
plt.imshow(proba_class1.transpose(), origin="lower",  extent=[tx.min(),tx.max(),ty.min(),ty.max()])
plt.axis('equal')
plt.plot(X[I,0], X[I,1], '.')
plt.plot(X[J,0], X[J,1], '.')
plt.axis('off');

## Exercice 3 : Penalized Regression

In [ ]:
help(LogisticRegression)
help(LinearRegression)

**Q: Dans l'aide, quel est l'argument qui permet de gérer la pénalité utilisée ?**

**Q: [Regression] Comparer les différentes pénalisations en termes de performances sur l'ensemble d'apprentissage et de test ainsi que sur l'estimation des coefficients. Commenter.**

**Q: [Classification] Comparer les différentes pénalisations en termes de performances sur l'ensemble d'apprentissage et de test ainsi que sur l'estimation des coefficients. Commenter.**

**Q: Créer une procédure d'optimisation des hyper-paramètres.**

## Exercice 4 : Decision Tree and random Forest

### Arbre de décision

The DecisionTreeClassifier() of the library ‘tree’ implements the decision tree for classification.

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

**Q: Quels sont les hyper-paramètres d'un arbre de décision ? Quelless sont les valeurs par défaut de `DecisionTreeClassifier()`?**

**Q: Calculer la prédiction des classes associées à toutes les entrées de l'ensemble des données de la grille et visualiser les frontières de décision.**

In [ ]:
from sklearn.tree import export_text
r = export_text(treefit); # treefit -> Model
print(r)

**Q: Que fait le code ci-dessus ?**

**Q: Créer une procédure d'optimisation des hyper-paramètres.**

### Forêts aléatoires

In [ ]:
from sklearn.ensemble import RandomForestClassifier

**Q: Quels sont les hyper-paramètres d'un arbre de décision ? Quelles sont les valeurs par défaut de `DecisionTreeClassifier()`?**

**Q: Calculer la prédiction des classes associées à toutes les entrées de l'ensemble de données de la grille et visualiser les frontières de décision.**

**Q: Créer une procédure d'optimisation des hyper-paramètres.**

## Exercice 5

**Q: Appliquer tout ce que vous avez vu au cours de ce TP au dataset [Maternal Health Risk](https://archive.ics.uci.edu/dataset/863/maternal+health+risk).**